# Custom MLflow Python Function 
### `ESM2 Protein Embedding Transformer`


<!-- # external ref https://huggingface.co/blog/AmelieSchreiber/protein-optimization-and-design -->

In [0]:
import sys
print(sys.version)

3.11.0rc1 (main, Aug 12 2022, 10:02:14) [GCC 11.2.0]


In [0]:
%sh python --version

Python 3.11.0rc1


In [0]:
# Create the requirements.txt file
requirements = """
torch==2.1.0
transformers==4.34.0
accelerate==0.23.0
cloudpickle==3.1.1
"""

requirements_path = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/requirements.txt"

# Save the requirements.txt file to UC volumes
with open(requirements_path, "w") as f:
    f.write(requirements)

In [0]:
%pip install -r {requirements_path}

  Obtaining dependency information for cloudpickle==3.1.1 from https://files.pythonhosted.org/packages/7e/e8/64c37fadfc2816a7701fa8a6ed8d87327c7d54eacfbfb6edab14a2f2be75/cloudpickle-3.1.1-py3-none-any.whl.metadata
  Attempting uninstall: cloudpickle
    Found existing installation: cloudpickle 2.2.1
    Not uninstalling cloudpickle at /databricks/python3/lib/python3.11/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-ca8f051f-dfb3-4cb5-9494-4bed415b3bc6
    Can't uninstall 'cloudpickle'. No files were found to uninstall.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython() 

In [0]:
catalog_name = "mmt_demos"  
schema_name = "dependencies"  
volume_name = "esm_artifacts"  
requirements_path = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/requirements.txt"

In [0]:
ESMWrapper: Custom wrapper class for the transformer model.
load_context: Loads the model and tokenizer from the provided artifacts and sets up the device (CPU or GPU).
predict: Takes input sequences, tokenizes them, runs them through the model, and returns the embeddings.

In [0]:
# Custom PyFunc wrapper for a large transformer model
import mlflow
import torch
from transformers import AutoModelForMaskedLM, AutoTokenizer, logging
import shutil
import pandas as pd
from mlflow.models.signature import infer_signature

# Set the logging level to ERROR to disable verbose messages
# logging.set_verbosity_error()

# Define a Custom wrapper class for the transformer model
class ESMWrapper(mlflow.pyfunc.PythonModel):
    # load_context: Loads the model and tokenizer from the provided artifacts and sets up the device (CPU or GPU).
    def load_context(self, context):
        # Load ESM model from saved files in the artifact
        # self.artifacts = context.artifacts

        # Determine the device to use (GPU if available, otherwise CPU)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # Load the tokenizer from the provided artifacts
        self.tokenizer = AutoTokenizer.from_pretrained(context.artifacts["tokenizer"])
        # Load the model from the provided artifacts
        self.model = AutoModelForMaskedLM.from_pretrained(context.artifacts["model"])
        # Move the model to the appropriate device (GPU or CPU)
        self.model.to(self.device)
        # Set the model to evaluation mode
        self.model.eval()

        # Set special tokens if they are not set; ensure that the beginning-of-sequence (bos_token) and separator (sep_token) tokens are set. If they are not already set, they are assigned the value of the cls_token (classification token).
        if self.tokenizer.bos_token is None:
            self.tokenizer.bos_token = self.tokenizer.cls_token
        if self.tokenizer.sep_token is None:
            self.tokenizer.sep_token = self.tokenizer.cls_token
    
    # Define the predict function which takes input sequences, tokenizes them, runs them through the model, and returns the embeddings
    def predict(self, context, model_input):
        protein_sequences = model_input["sequences"]
        results = []
        
        # Process each sequence
        for seq in protein_sequences:
            inputs = self.tokenizer(seq, return_tensors="pt")
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            
            with torch.no_grad():
                outputs = self.model(**inputs, output_hidden_states=True)
                
            # Process outputs as needed for your application
            embeddings = outputs.hidden_states[-1].mean(dim=1).cpu().numpy()
            results.append(embeddings)
            
        return results

# Log the model with explicit dependencies
with mlflow.start_run():
    # Download and Save Model Components:
    model_name = "facebook/esm2_t33_650M_UR50D" #https://huggingface.co/facebook/esm2_t33_650M_UR50D
    
    # Specifies the model name and paths to save the model and tokenizer.
    model_path = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/tmp_model"
    tokenizer_path = f"/Volumes/{catalog_name}/{schema_name}/{volume_name}/tmp_tokenizer"
    
    # Download the pre-trained model and tokenizer from Hugging Face.
    model = AutoModelForMaskedLM.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Saves the model and tokenizer to the specified paths.
    model.save_pretrained(model_path, safe_serialization=False)
    tokenizer.save_pretrained(tokenizer_path)
    
    # Define conda env with necessary dependencies
    conda_env = {
        "channels": ["defaults", "conda-forge", "pytorch"],
        "dependencies": [
            "python=3.11", # compute 15.4LTSMLR ##"python=3.10",
            "pip>=22.0.4",
            {"pip": [
                "torch==2.1.0", 
                "transformers==4.34.0",
                "accelerate==0.23.0",
                "cloudpickle==3.1.1", #compute 15.4LTSMLR  ##"cloudpickle==2.2.1"  
            ]}
        ],
        "name": "esm_env"
    }
    
    # Create a sample input DataFrame to infer the input and output signature of the model.
    sample_input = pd.DataFrame({"sequences": ["MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG"]})
    
    ## Initialize the wrapper and load the context manually for signature inference

    # Initialize an instance of the ESMWrapper class.
    esm_wrapper = ESMWrapper()
    # Manually set the tokenizer and model for the wrapper.
    esm_wrapper.tokenizer = tokenizer
    esm_wrapper.model = model
    # Determine the device (GPU or CPU) and moves the model to the appropriate device.
    esm_wrapper.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    esm_wrapper.model.to(esm_wrapper.device)
    # Set the model to evaluation mode.
    esm_wrapper.model.eval()
    
    # Use the wrapper to predict the output for the sample input
    sample_output = esm_wrapper.predict(None, sample_input)
    # Infer the input and output signature of the model using the sample input and output
    signature = infer_signature(sample_input, sample_output)
    
    # Log the model with MLflow, including the artifacts (model and tokenizer paths), Conda environment, signature, and input example.
    mlflow.pyfunc.log_model(
        artifact_path="esm_model",
        python_model=ESMWrapper(),
        artifacts={
            "model": model_path,
            "tokenizer": tokenizer_path,
            "requirements": requirements_path  
        },
        conda_env=conda_env,
        signature=signature,
        input_example=sample_input,
        # Register the model with a specified name.
        registered_model_name=f"{catalog_name}.{schema_name}.esm_protein_model"
    )

Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.


Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.


Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.


Uploading artifacts:   0%|          | 0/18 [00:00<?, ?it/s]

Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.


...

Uploading artifacts:   0%|          | 0/18 [00:00<?, ?it/s]

Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.


Uploading /local_disk0/repl_tmp_data/ReplId-19610-f2e2d-7/tmpc10avx6k/model/artifacts/tmp_model/pytorch_model.…

Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
Created version '8' of model 'mmt_demos.dependencies.esm_protein_model'.
Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.


🏃 View run adorable-cat-10 at: https://e2-demo-field-eng.cloud.databricks.com/ml/experiments/198632f0fb56423385dbfbb273091fa0/runs/4b068990e6814a4ba481240f038a5fef
🧪 View experiment at: https://e2-demo-field-eng.cloud.databricks.com/ml/experiments/198632f0fb56423385dbfbb273091fa0


In [0]:
sample_output

[array([[ 0.06840784,  0.06890779, -0.04763587, ..., -0.0151379 ,
         -0.1210696 ,  0.03015589]], dtype=float32)]

In [0]:
# [9xz5z] [2025-04-07 16:15:05 +0000] 2025/04/07 16:15:05 WARNING mlflow.pyfunc: The version of CloudPickle that was used to save the model, `CloudPickle 2.2.1`, differs from the version of CloudPickle that is currently running, `CloudPickle 3.1.1`, and may be incompatible

In [0]:
import mlflow

model_uri = f"models:/{catalog_name}.{schema_name}.esm_protein_model/8"
dependencies = mlflow.pyfunc.get_model_dependencies(model_uri)
print(dependencies)

2025/04/07 22:59:17 INFO mlflow.pyfunc: To install the dependencies that were used to train the model, run the following command: '%pip install -r /local_disk0/repl_tmp_data/ReplId-19610-f2e2d-7/tmpg3ihn9ch/requirements.txt'.


/local_disk0/repl_tmp_data/ReplId-19610-f2e2d-7/tmpg3ihn9ch/requirements.txt


In [0]:
!cat /local_disk0/repl_tmp_data/ReplId-19610-f2e2d-7/tmpg3ihn9ch/requirements.txt

mlflow==2.19.0
torch==2.1.0
transformers==4.34.0
accelerate==0.23.0
cloudpickle==3.1.1

In [0]:
from mlflow.tracking import MlflowClient

def get_latest_model_version(model_name):
    mlflow_client = MlflowClient(registry_uri="databricks-uc")
    latest_version = 1
    for mv in mlflow_client.search_model_versions(f"name='{model_name}'"):
        version_int = int(mv.version)
        if version_int > latest_version:
            latest_version = version_int
    return latest_version

def get_model_uri(model_name):
  return f"models:/{model_name}/{get_latest_model_version(model_name)}"

In [0]:
host = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

workload_type = "GPU_SMALL"
workload_size = "Small"

registered_model_name = f"{catalog_name}.{schema_name}.esm_protein_model"
latest_model_version = get_latest_model_version(registered_model_name)
general_model_name = f"esm_protein_model-{latest_model_version}" 

endpoint_base_name = "esm_protein_model_endpoint"
endpoint_name = f"{endpoint_base_name}-mmt-mlflowsdk" # endpoint name

print(f"catalog: {catalog_name}",  "schema:", {schema_name}, "host:", host, "token:", token, workload_type, workload_size, "general_model_name:", general_model_name, "registered_model_name:", registered_model_name, "latest_model_version:", latest_model_version, "endpoint_name:", endpoint_name)

catalog: mmt_demos schema: {'dependencies'} host: https://oregon.cloud.databricks.com token: [REDACTED] GPU_SMALL Small general_model_name: esm_protein_model-8 registered_model_name: mmt_demos.dependencies.esm_protein_model latest_model_version: 8 endpoint_name: esm_protein_model_endpoint-mmt-mlflowsdk


In [0]:
# client.delete_endpoint(endpoint_name)

{}

In [0]:
from mlflow.deployments import get_deploy_client

client = get_deploy_client("databricks")

# Define the full API request payload
endpoint_config = {
    "name": general_model_name,
    "served_models": [
        {                
            "model_name": registered_model_name,
            "model_version": latest_model_version,
            "workload_size": workload_size,  # defines concurrency: Small/Medium/Large
            "workload_type": workload_type,  # defines compute: GPU_SMALL/GPU_MEDIUM/GPU_LARGE
            "scale_to_zero_enabled": True
        }
    ],
    "traffic_config": {
        "routes": [
            {
                "served_model_name": general_model_name,
                "traffic_percentage": 100
            }
        ]
    },
    "auto_capture_config": {
        "catalog_name": catalog_name,
        "schema_name": schema_name,
        # "table_name_prefix": endpoint_base_name,
        "enabled": True
    },
    "tags": {
        "project": "esm_protein_model",
        "team": "ml-team",
        "removeAfter": "2025-12-31",
    }
}

# Create or update the endpoint
try:
    # Check if endpoint exists
    existing_endpoint = client.get_endpoint(endpoint_name)
    print(f"Endpoint {endpoint_name} exists, updating configuration...")
    client.update_endpoint_config(endpoint_name, endpoint_config)
except Exception as e:
    if "RESOURCE_DOES_NOT_EXIST" in str(e):
        print(f"Creating new endpoint {endpoint_name}...")
        client.create_endpoint(endpoint_name, endpoint_config)
    else:
        raise

Endpoint esm_protein_model_endpoint-mmt-mlflowsdk exists, updating configuration...


/root/.ipykernel/33219/command-3394427869929427-3408967896:32: FutureWarning: ``mlflow.deployments.databricks.DatabricksDeploymentClient.update_endpoint`` is deprecated. This method will be removed in a future release. Use ``update_endpoint_config, update_endpoint_tags, update_endpoint_rate_limits, or update_endpoint_ai_gateway`` instead.
  client.update_endpoint(endpoint_name, endpoint_config)
/databricks/python/lib/python3.11/site-packages/mlflow/deployments/databricks/__init__.py:547: UserWarning: The `update_endpoint` method is deprecated. Use the specific update methods—`update_endpoint_config`, `update_endpoint_tags`, `update_endpoint_rate_limits`, `update_endpoint_ai_gateway`—instead.
  warnings.warn(


In [0]:
import requests
import json
import time

response = requests.post(
    f"{workspace_url}/api/2.0/serving-endpoints",
    headers={"Authorization": f"Bearer {token}"},
    data=json.dumps(endpoint_config)
)

if response.status_code == 200:
    print("Serving endpoint created successfully.")
else:
    print(f"Failed to create serving endpoint: {response.text}")

# Check the status of the endpoint
endpoint_status_url = f"{workspace_url}/api/2.0/serving-endpoints/{endpoint_name}"

max_retries = 6  # Number of times to check (6 times for 1 hour)
retry_interval = 600  # 10 minutes in seconds

# while True:
for attempt in range(max_retries):

    status_response = requests.get(
        endpoint_status_url,
        headers={"Authorization": f"Bearer {token}"}
    )
    if status_response.status_code == 200:
        state = status_response.json().get("state", {})
        ready = state.get("ready", False)
        config_update = state.get("config_update", False)
        message = state.get("message", "No status message available")
        pending_config = status_response.json().get('pending_config', {})
        served_entities = pending_config.get('served_entities', [{}])
        served_entity_state = served_entities[0].get('state', 'Unknown')
        
        if ready and config_update != 'IN_PROGRESS':
            print("Serving endpoint is ready.")
            break
        else:
            print(state)
            print(f"Serving endpoint state: {message}")
            print(f"Served entity state: {served_entity_state}")
            print("Waiting for the serving endpoint config_updates to be ready...")
            time.sleep(retry_interval)  # Wait for 10 minutes before checking again
    else:
        print(f"Failed to get endpoint status: {status_response.text}")
        break
else:
    print("Endpoint config_update is still not ready after the maximum number of check retries. Please review the Serving UI to see if there are errors or issues.")

Failed to create serving endpoint: {"error_code":"RESOURCE_ALREADY_EXISTS","message":"Endpoint with name 'esm_protein_model-8' already exists.","details":[{"@type":"type.googleapis.com/google.rpc.RequestInfo","request_id":"4d6f7f20-d790-4f9d-b62b-f354ce367af6","serving_data":""}]}
{'ready': 'READY', 'config_update': 'IN_PROGRESS', 'suspend': 'NOT_SUSPENDED'}
Serving endpoint state: No status message available
Served entity state: {'deployment': 'DEPLOYMENT_CREATING', 'deployment_state_message': 'Provisioning resources for served entities.'}
Waiting for the serving endpoint config_updates to be ready...
Serving endpoint is ready.


In [0]:
status_response = requests.get(
        endpoint_status_url,
        headers={"Authorization": f"Bearer {token}"}
    )

status_response.json()

{'name': 'esm_protein_model_endpoint-mmt-mlflowsdk',
 'creator': 'may.merkletan@databricks.com',
 'creation_timestamp': 1744049525000,
 'last_updated_timestamp': 1744057181000,
 'state': {'ready': 'READY',
  'config_update': 'IN_PROGRESS',
  'suspend': 'NOT_SUSPENDED'},
 'config': {'served_entities': [{'name': 'esm_protein_model-6',
    'entity_name': 'mmt_demos.dependencies.esm_protein_model',
    'entity_version': '6',
    'workload_size': 'Small',
    'workload_type': 'GPU_SMALL',
    'scale_to_zero_enabled': True,
    'state': {'deployment': 'DEPLOYMENT_READY',
     'deployment_state_message': 'Scaled to zero'},
    'creator': 'may.merkletan@databricks.com',
    'creation_timestamp': 1744049525000}],
  'served_models': [{'name': 'esm_protein_model-6',
    'workload_size': 'Small',
    'workload_type': 'GPU_SMALL',
    'scale_to_zero_enabled': True,
    'model_name': 'mmt_demos.dependencies.esm_protein_model',
    'model_version': '6',
    'state': {'deployment': 'DEPLOYMENT_READY',

In [0]:
state = status_response.json().get("state", {})
state
# ready = state.get("ready", False)

{'ready': 'READY', 'config_update': 'IN_PROGRESS', 'suspend': 'NOT_SUSPENDED'}

In [0]:
(status_response.json()['state'], 
 status_response.json()['state']['ready'],
 status_response.json()['pending_config']['served_entities'][0]['state']
 )

({'ready': 'READY',
  'config_update': 'IN_PROGRESS',
  'suspend': 'NOT_SUSPENDED'},
 'READY',
 {'deployment': 'DEPLOYMENT_CREATING',
  'deployment_state_message': 'Creating resources for served entity.'})

In [0]:
import requests
import json

# Extract Databricks workspace URL and token using dbutils
workspace_url = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

# Endpoint URL
endpoint_url = f"{workspace_url}/serving-endpoints/{endpoint_name}/invocations"

# Sample input
input_data = {
    "dataframe_records": [
        {"sequences": "MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG"}
    ]
}

response = requests.post(
    endpoint_url,
    headers={"Authorization": f"Bearer {token}", "Content-Type": "application/json"},
    data=json.dumps(input_data)
)

if response.status_code == 200:
    print("Prediction:", response.json())
else:
    print(f"Failed to get prediction: {response.text}")

Prediction: {'predictions': [[[0.06840793788433075, 0.06890784204006195, -0.04763609543442726, 0.02628912404179573, -0.08527260273694992, 0.039011821150779724, -0.0711316242814064, 0.20650677382946014, 0.07555650174617767, 0.02792150154709816, 0.010508893989026546, 0.05739198997616768, 0.14937055110931396, 0.13881385326385498, -0.05469195544719696, 0.06682949513196945, -0.016842197626829147, 0.04010609909892082, -0.057004936039447784, -0.06419392675161362, 0.08771298080682755, -0.03626326099038124, -0.00040011206874623895, 0.08976373821496964, 0.043842874467372894, -0.05397772043943405, -0.06927525997161865, -0.06238190084695816, -0.05292431637644768, -0.15746259689331055, -0.05517962947487831, -0.056654464453458786, 0.07234666496515274, -0.032119181007146835, -0.012918336316943169, 0.006699382793158293, -0.10783190280199051, 0.032689131796360016, 0.09280376881361008, -0.023149622604250908, 0.035779647529125214, 0.05187513679265976, 0.10416208207607269, 0.03620433807373047, -0.02049941

In [0]:
input_data = {
    "dataframe_records": [
        {"sequences": "MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG"}
    ]
}

input_data

{'dataframe_records': [{'sequences': 'MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG'}]}

In [0]:
json.dumps(input_data)

'{"dataframe_records": [{"sequences": "MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG"}]}'

In [0]:
# https://huggingface.co/blog/AmelieSchreiber/protein-optimization-and-design 

# MAPLRKTYVLKLYVAGNTPNSVRALKTLNNILEKEFKGVYALKVIDVLKNPQLAEEDKILATPTLAKVLPPPVRRIIGDLSNREKVLIGLDLLYEEIGDQAEDDLGLE

In [0]:
input_data2 = {
    "dataframe_records": [
        {"sequences": "MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG"},
        {"sequences": "MAPLRKTYVLKLYVAGNTPNSVRALKTLNNILEKEFKGVYALKVIDVLKNPQLAEEDKILATPTLAKVLPPPVRRIIGDLSNREKVLIGLDLLYEEIGDQAEDDLGLE"}
    ]
}

input_data2, json.dumps(input_data2)

In [0]:
import requests
import json

# Extract Databricks workspace URL and token using dbutils
workspace_url = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiUrl().get()
token = dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

# Endpoint URL
endpoint_url = f"{workspace_url}/serving-endpoints/{endpoint_name}/invocations"

# Sample input with multiple sequences
input_data = {
    "dataframe_records": [
        {"sequences": "MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG"},
        {"sequences": "MAPLRKTYVLKLYVAGNTPNSVRALKTLNNILEKEFKGVYALKVIDVLKNPQLAEEDKILATPTLAKVLPPPVRRIIGDLSNREKVLIGLDLLYEEIGDQAEDDLGLE"}
    ]
}

response = requests.post(
    endpoint_url,
    headers={"Authorization": f"Bearer {token}", "Content-Type": "application/json"},
    data=json.dumps(input_data)
)

if response.status_code == 200:
    print("Prediction:", response.json())
else:
    print(f"Failed to get prediction: {response.text}")

Prediction: {'predictions': [[[0.06840793788433075, 0.06890784204006195, -0.04763609543442726, 0.02628912404179573, -0.08527260273694992, 0.039011821150779724, -0.0711316242814064, 0.20650677382946014, 0.07555650174617767, 0.02792150154709816, 0.010508893989026546, 0.05739198997616768, 0.14937055110931396, 0.13881385326385498, -0.05469195544719696, 0.06682949513196945, -0.016842197626829147, 0.04010609909892082, -0.057004936039447784, -0.06419392675161362, 0.08771298080682755, -0.03626326099038124, -0.00040011206874623895, 0.08976373821496964, 0.043842874467372894, -0.05397772043943405, -0.06927525997161865, -0.06238190084695816, -0.05292431637644768, -0.15746259689331055, -0.05517962947487831, -0.056654464453458786, 0.07234666496515274, -0.032119181007146835, -0.012918336316943169, 0.006699382793158293, -0.10783190280199051, 0.032689131796360016, 0.09280376881361008, -0.023149622604250908, 0.035779647529125214, 0.05187513679265976, 0.10416208207607269, 0.03620433807373047, -0.02049941

In [0]:
## actually slower... 

import mlflow.pyfunc
from pyspark.sql.functions import col, flatten

# Load the registered model as a Spark UDF
model_name = "mmt_demos.dependencies.esm_protein_model"
model_version = 8
model_uri = f"models:/{model_name}/{model_version}"
model_udf = mlflow.pyfunc.spark_udf(spark, model_uri=model_uri)

# Example Spark DataFrame with sequences
df = spark.createDataFrame([
    ("MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG",),
    ("MAPLRKTYVLKLYVAGNTPNSVRALKTLNNILEKEFKGVYALKVIDVLKNPQLAEEDKILATPTLAKVLPPPVRRIIGDLSNREKVLIGLDLLYEEIGDQAEDDLGLE",)
], ["sequences"])

# Apply the model UDF to the DataFrame
df_result = df.withColumn("predictions", 
                          flatten(model_udf(col("sequences"))) 
                         )

# Display the result
display(df_result)

Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.


Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
2025/04/08 01:57:03 WARNING mlflow.pyfunc: Calling `spark_udf()` with `env_manager="local"` does not recreate the same environment that was used during training, which may lead to errors or inaccurate predictions. We recommend specifying `env_manager="conda"`, which automatically recreates the environment that was used to train the model and performs inference in the recreated environment.


Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.
2025/04/08 01:57:06 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
Using bos_token, but it is not set yet.
Using sep_token, but it is not set yet.


sequences predictions MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG List(0.06840784, 0.06890779, -0.04763587, 0.026289457, -0.08527259, 0.039011963, -0.07113155, 0.2065064, 0.075556435, 0.02792131, 0.010508892, 0.057391778, 0.14937049, 0.13881369, -0.05469166, 0.06682947, -0.016841978, 0.040105898, -0.05700479, -0.06419369, 0.08771316, -0.036263123, -3.9970677E-4, 0.08976342, 0.04384283, -0.05397747, -0.06927493, -0.062382277, -0.05292437, -0.15746267, -0.055179078, -0.05665435, 0.072346784, -0.032119017, -0.012918629, 0.006699154, -0.10783194, 0.03268915, 0.09280367, -0.023149654, 0.03577979, 0.051875185, 0.104161836, 0.036204383, -0.020498917, -0.13204816, -0.0252537, 0.03561942, 0.0059802164, -0.055135865, -2.1757484E-4, 0.019700207, -0.023576146, -0.009352228, -0.059250224, 0.071285196, 0.050104473, 0.12734555, -0.05880805, -0.09344801, 0.09654925, 0.059749074, -0.042374797, -0.040041715, -0.0013047033, 0.012205678, 0.12572226, -0.14124309, -0.15539794, -0.06160593, 0.01936284, -0.08097414, -0.03506948, 0.07391897, -0.040058482, -0.059401155, -0.0891893, 0.060319316, -0.020221105, -0.005621419, 0.04852418, 0.018279202, 0.0010251909, 0.09307838, 0.027128352, -0.09231574, -0.09629831, 0.04284475, -0.018366126, 0.04809984, -0.029143998, -0.034994457, -0.17611793, 0.010257699, -0.088485144, -0.043884598, 0.10064366, -0.033974245, -0.11548829, -0.036989152, 0.043327715, 0.036029894, 0.13751398, -0.11825176, -0.038718514, 0.036287233, -0.14142641, 0.032923322, -0.03525827, -0.0018855178, -0.14186998, 0.05842803, -0.0035115222, -0.08143006, 0.019648788, 0.11413247, -0.055711504, -0.08016897, -0.0075733764, 0.16955669, 0.12441386, -0.017836567, 0.026962413, 0.05134395, -0.089070514, -0.041796885, 0.04541226, 0.035128776, -0.12006121, 0.050877616, 0.069432326, -0.028864633, -0.05821075, 0.07436316, 0.04504408, 0.07949482, -0.066598356, -0.15419696, 0.03215664, -0.02038155, 0.049383335, 0.15750231, -0.11478375, 0.049876563, -0.01771551, 0.0934462, 0.019007105, -2.2472806E-4, -0.061440844, 0.097847424, 0.004283445, 0.05370673, 0.009988255, -0.1501442, 0.14166494, 0.097513966, 0.102532715, 0.0053088237, 0.13271771, -0.05987821, 0.022199856, 0.11713531, -0.011334739, -0.024710203, -0.023027718, -0.016130878, -0.046666887, 0.014623913, 0.08421318, 0.06692704, 0.025923843, 0.022273125, 0.06477591, -0.0113320025, -0.08613913, 0.050319817, -0.00413663, -0.044563368, 0.15151925, -0.017016342, -0.08614613, -0.0768333, 0.020288527, 0.0017804999, 0.019006718, -0.011663687, 0.032768667, -0.10095035, 0.009795879, 0.12488068, 0.00837066, -0.02633033, -0.0040276535, -0.021209532, 0.039961472, 0.010805586, -0.02543691, -0.009648451, -0.025574759, 0.0133662475, 0.060807765, 0.051675566, -0.11963525, 0.017710317, 0.047936212, -0.04474188, 0.114512965, 0.00992879, -0.12453615, -0.125669, 0.0965102, -0.0019429479, 0.013614233, -0.02550386, -0.10239068, -0.06484313, 0.19323383, 0.04834419, 0.10714169, 0.03091554, -0.041921206, 0.06626287, 0.06526202, -0.026423516, 0.05881859, -0.026068816, 0.15361182, 0.17265049, -0.19219702, -0.049139846, 0.07328678, -0.01387159, -0.1120434, 0.0029987283, -4.790729, 0.028447099, 0.01568717, -0.08928923, 0.052843764, -0.25598755, 0.017638419, 0.047088612, -8.846339E-4, 0.032548886, -0.0013406181, -0.08099665, 0.014094312, 0.05428161, 0.008895334, -0.052127156, -0.019124053, -0.031808704, -0.051905144, 0.09669104, 0.1698379, 0.0196403, 0.015870437, 0.05906175, -0.046348765, -0.10003106, 0.106872775, 0.0067229043, -0.019107854, 0.10458531, 0.15299396, -0.0064429445, -0.083470784, 0.040234607, -0.10229632, -0.029127296, 0.010930046, -0.057262175, 0.01847434, 0.019283928, 0.07076579, 0.14076078, -0.062730186, -0.108375974, -0.047029145, -0.1143748, -0.028760273, -0.21158278, 0.043966733, 0.020461867, -0.1051079, -0.1648521, 0.065569654, -0.008672121, -5.863448E-4, 0.029037243, -0.12931879, -0.007842298, -0.16524561, -0.020396343, -0.066592485, 0.01604696, -0.032112647, -0.040887292, 0.113839, -0.005290776, -0.0025289436, 0.04

In [0]:
endpoint_name
# esm_protein_model_endpoint-mmt-mlflowsdk

'esm_protein_model_endpoint-mmt-mlflowsdk'

In [0]:
import os
import requests
import numpy as np
import pandas as pd
import json

def get_databricks_token():
    return dbutils.notebook.entry_point.getDbutils().notebook().getContext().apiToken().get()

def create_tf_serving_json(data):
    return {'inputs': {name: data[name].tolist() for name in data.keys()} if isinstance(data, dict) else data.tolist()}

def score_model(dataset):
    url = f'https://e2-demo-field-eng.cloud.databricks.com/serving-endpoints/{endpoint_name}/invocations'
    token = get_databricks_token()
    headers = {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json'}
    ds_dict = {'dataframe_split': dataset.to_dict(orient='split')} if isinstance(dataset, pd.DataFrame) else create_tf_serving_json(dataset)
    data_json = json.dumps(ds_dict, allow_nan=True)
    response = requests.request(method='POST', headers=headers, url=url, data=data_json)
    if response.status_code != 200:
        raise Exception(f'Request failed with status {response.status_code}, {response.text}')
    return response.json()

In [0]:
import pandas as pd
from pyspark.sql.functions import col

# Example Spark DataFrame with sequences
df = spark.createDataFrame([
    ("MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG",),
    ("MAPLRKTYVLKLYVAGNTPNSVRALKTLNNILEKEFKGVYALKVIDVLKNPQLAEEDKILATPTLAKVLPPPVRRIIGDLSNREKVLIGLDLLYEEIGDQAEDDLGLE",)
], ["sequences"])

# Convert Spark DataFrame to Pandas DataFrame for batch inferencing
df_pd = df.toPandas()

# Perform batch inferencing using the score_model function
model_score = score_model(df_pd)

# Convert predictions back to Spark DataFrame
predictions_df = pd.DataFrame(model_score['predictions'])
scored_df = spark.createDataFrame(predictions_df)

# Display the scored DataFrame
display(scored_df)

0


In [0]:
df_pd

,sequences
0,MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG
1,MAPLRKTYVLKLYVAGNTPNSVRALKTLNNILEKEFKGVYALKVID...


In [0]:
ds_dict = {'dataframe_split': df_pd.to_dict(orient='split')} if isinstance(df_pd, pd.DataFrame) else create_tf_serving_json(df_pd)

In [0]:
ds_dict

{'dataframe_split': {'index': [0, 1],
  'columns': ['sequences'],
  'data': [['MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG'],
   ['MAPLRKTYVLKLYVAGNTPNSVRALKTLNNILEKEFKGVYALKVIDVLKNPQLAEEDKILATPTLAKVLPPPVRRIIGDLSNREKVLIGLDLLYEEIGDQAEDDLGLE']]}}

In [0]:
data_json = json.dumps(ds_dict, allow_nan=True)

In [0]:
data_json

'{"dataframe_split": {"index": [0, 1], "columns": ["sequences"], "data": [["MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG"], ["MAPLRKTYVLKLYVAGNTPNSVRALKTLNNILEKEFKGVYALKVIDVLKNPQLAEEDKILATPTLAKVLPPPVRRIIGDLSNREKVLIGLDLLYEEIGDQAEDDLGLE"]]}}'

In [0]:
import os
import requests
import numpy as np
import pandas as pd
import json
from pyspark.sql.functions import pandas_udf, col, explode
from pyspark.sql.types import ArrayType, FloatType, StringType, StructType, StructField

# Set your Personal Access Token (PAT) here
PAT_TOKEN = dbutils.secrets.get(scope="mmt", key="databricks_token")

def create_tf_serving_json(data):
    return {'inputs': {name: data[name].tolist() for name in data.keys()} if isinstance(data, dict) else data.tolist()}

def score_model(dataset):
    url = f'https://e2-demo-field-eng.cloud.databricks.com/serving-endpoints/{endpoint_name}/invocations'
    token = PAT_TOKEN
    headers = {'Authorization': f'Bearer {token}', 'Content-Type': 'application/json'}
    ds_dict = {'dataframe_split': dataset.to_dict(orient='split')} if isinstance(dataset, pd.DataFrame) else create_tf_serving_json(dataset)
    data_json = json.dumps(ds_dict, allow_nan=True)
    response = requests.request(method='POST', headers=headers, url=url, data=data_json)
    if response.status_code != 200:
        raise Exception(f'Request failed with status {response.status_code}, {response.text}')
    return response.json()['predictions']

# Define the schema for the output
schema = StructType([
    StructField("sequences", StringType(), True),
    StructField("predictions", ArrayType(FloatType()), True)
])

# Define the Pandas UDF
@pandas_udf(schema)
def score_model_udf(sequences: pd.Series) -> pd.DataFrame:
    results = []
    for sequence in sequences:
        predictions = score_model(pd.DataFrame({"sequences": [sequence]}))[0]
        # Flatten the predictions if they are nested lists
        if isinstance(predictions[0], list):
            scores = [item for sublist in predictions for item in sublist]
        scores = np.array(scores).astype(np.float32).tolist()  # Convert to float32 and then to list
        results.append({"sequences": sequence, "predictions": scores})
    return pd.DataFrame(results)

# Example Spark DataFrame with sequences
df = spark.createDataFrame([
    ("MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG",),
    ("MAPLRKTYVLKLYVAGNTPNSVRALKTLNNILEKEFKGVYALKVIDVLKNPQLAEEDKILATPTLAKVLPPPVRRIIGDLSNREKVLIGLDLLYEEIGDQAEDDLGLE",)
], ["sequences"])

# Apply the Pandas UDF to the DataFrame
scored_df = df.withColumn("scores", score_model_udf(col("sequences")))


In [0]:
## quite fast

display(scored_df)

sequences scores MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG List(MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG, List(0.06840794, 0.06890784, -0.047636095, 0.026289124, -0.0852726, 0.03901182, -0.071131624, 0.20650677, 0.0755565, 0.027921502, 0.010508894, 0.05739199, 0.14937055, 0.13881385, -0.054691955, 0.066829495, -0.016842198, 0.0401061, -0.057004936, -0.06419393, 0.08771298, -0.03626326, -4.0011207E-4, 0.08976374, 0.043842874, -0.05397772, -0.06927526, -0.0623819, -0.052924316, -0.1574626, -0.05517963, -0.056654464, 0.072346665, -0.03211918, -0.012918336, 0.006699383, -0.1078319, 0.03268913, 0.09280377, -0.023149623, 0.035779648, 0.051875137, 0.10416208, 0.036204338, -0.020499412, -0.13204803, -0.02525354, 0.03561925, 0.0059801224, -0.055135846, -2.1745828E-4, 0.019699689, -0.023576265, -0.00935201, -0.059250455, 0.071284935, 0.050104868, 0.12734553, -0.05880816, -0.09344792, 0.096549176, 0.059749223, -0.042374775, -0.040042143, -0.0013046728, 0.012205901, 0.12572211, -0.14124347, -0.1553979, -0.061606538, 0.019362643, -0.08097422, -0.035069484, 0.07391929, -0.040058672, -0.059401195, -0.08918937, 0.06031931, -0.020221155, -0.0056216037, 0.048523802, 0.018279234, 0.0010251419, 0.09307883, 0.027128117, -0.09231592, -0.09629823, 0.042844374, -0.018366234, 0.04810005, -0.02914399, -0.03499469, -0.17611836, 0.010257617, -0.08848514, -0.043884527, 0.100643806, -0.033974346, -0.1154887, -0.0369891, 0.04332762, 0.03602978, 0.13751431, -0.11825207, -0.03871867, 0.03628723, -0.14142665, 0.032923404, -0.03525817, -0.001885201, -0.14187008, 0.05842831, -0.0035118521, -0.08143016, 0.019648846, 0.11413273, -0.05571138, -0.08016915, -0.0075730034, 0.16955711, 0.12441398, -0.017836358, 0.026962243, 0.051343814, -0.089070395, -0.04179708, 0.045412768, 0.035129357, -0.12006119, 0.05087762, 0.069432355, -0.02886489, -0.058211014, 0.074363135, 0.045044165, 0.0794948, -0.06659776, -0.15419686, 0.032156974, -0.020381687, 0.049383298, 0.15750237, -0.11478398, 0.0498765, -0.01771576, 0.093446225, 0.019006833, -2.2464048E-4, -0.061441038, 0.097847626, 0.004283281, 0.053706992, 0.009988454, -0.15014431, 0.14166535, 0.09751435, 0.102532476, 0.005308849, 0.13271774, -0.059878107, 0.0221993, 0.117135644, -0.011334801, -0.024710264, -0.023027897, -0.016130574, -0.046666972, 0.014624121, 0.08421323, 0.06692702, 0.025923515, 0.0222732, 0.06477591, -0.0113320025, -0.08613905, 0.050319705, -0.004136385, -0.044563565, 0.1515195, -0.017016126, -0.0861463, -0.07683315, 0.020288685, 0.0017801934, 0.019006534, -0.011663531, 0.032768846, -0.100950435, 0.009796022, 0.124880776, 0.008370491, -0.026330657, -0.004027821, -0.021209696, 0.039961133, 0.010805228, -0.025436739, -0.009648199, -0.025574747, 0.013366432, 0.06080755, 0.051675554, -0.11963538, 0.01771051, 0.047935985, -0.04474174, 0.11451311, 0.009928426, -0.12453619, -0.12566896, 0.096510224, -0.001942677, 0.013614137, -0.025503954, -0.10239078, -0.06484347, 0.193234, 0.04834418, 0.10714206, 0.03091593, -0.041921344, 0.06626271, 0.065262206, -0.026423577, 0.058818113, -0.026069207, 0.15361166, 0.17265041, -0.19219702, -0.04913946, 0.073286936, -0.013871239, -0.11204348, 0.0029985793, -4.790734, 0.028446784, 0.015687099, -0.08928978, 0.05284358, -0.25598767, 0.017638793, 0.04708858, -8.8494027E-4, 0.032549053, -0.0013408883, -0.08099685, 0.014094401, 0.05428197, 0.008895735, -0.052127443, -0.019123822, -0.03180861, -0.05190555, 0.09669127, 0.16983789, 0.019639824, 0.01587091, 0.05906193, -0.046349242, -0.10003088, 0.10687288, 0.0067231646, -0.019107644, 0.10458537, 0.15299408, -0.006442845, -0.08347113, 0.040234894, -0.10229683, -0.029127132, 0.010930232, -0.05726208, 0.018474119, 0.019283976, 0.070765875, 0.14076112, -0.06273048, -0.108375944, -0.047028888, -0.1143751, -0.028760418, -0.21158293, 0.043967094, 0.020462122, -0.10510801, -0.16485217, 0.06556936, -0.008671817, -5.8623287E-4, 0.029037379, -0.1293192, -0.007842674, -0.16524564, -0.020396668, -0.06659272, 0.016046695, -0.032112647, -0.040887475, 0.

In [0]:
display(scored_df.select('scores.*'))

sequences predictions MKTAYIAKQRQISFVKSHFSRQDILDLWIYHTQGYFPDWQNYG List(0.06840794, 0.06890784, -0.047636095, 0.026289124, -0.0852726, 0.03901182, -0.071131624, 0.20650677, 0.0755565, 0.027921502, 0.010508894, 0.05739199, 0.14937055, 0.13881385, -0.054691955, 0.066829495, -0.016842198, 0.0401061, -0.057004936, -0.06419393, 0.08771298, -0.03626326, -4.0011207E-4, 0.08976374, 0.043842874, -0.05397772, -0.06927526, -0.0623819, -0.052924316, -0.1574626, -0.05517963, -0.056654464, 0.072346665, -0.03211918, -0.012918336, 0.006699383, -0.1078319, 0.03268913, 0.09280377, -0.023149623, 0.035779648, 0.051875137, 0.10416208, 0.036204338, -0.020499412, -0.13204803, -0.02525354, 0.03561925, 0.0059801224, -0.055135846, -2.1745828E-4, 0.019699689, -0.023576265, -0.00935201, -0.059250455, 0.071284935, 0.050104868, 0.12734553, -0.05880816, -0.09344792, 0.096549176, 0.059749223, -0.042374775, -0.040042143, -0.0013046728, 0.012205901, 0.12572211, -0.14124347, -0.1553979, -0.061606538, 0.019362643, -0.08097422, -0.035069484, 0.07391929, -0.040058672, -0.059401195, -0.08918937, 0.06031931, -0.020221155, -0.0056216037, 0.048523802, 0.018279234, 0.0010251419, 0.09307883, 0.027128117, -0.09231592, -0.09629823, 0.042844374, -0.018366234, 0.04810005, -0.02914399, -0.03499469, -0.17611836, 0.010257617, -0.08848514, -0.043884527, 0.100643806, -0.033974346, -0.1154887, -0.0369891, 0.04332762, 0.03602978, 0.13751431, -0.11825207, -0.03871867, 0.03628723, -0.14142665, 0.032923404, -0.03525817, -0.001885201, -0.14187008, 0.05842831, -0.0035118521, -0.08143016, 0.019648846, 0.11413273, -0.05571138, -0.08016915, -0.0075730034, 0.16955711, 0.12441398, -0.017836358, 0.026962243, 0.051343814, -0.089070395, -0.04179708, 0.045412768, 0.035129357, -0.12006119, 0.05087762, 0.069432355, -0.02886489, -0.058211014, 0.074363135, 0.045044165, 0.0794948, -0.06659776, -0.15419686, 0.032156974, -0.020381687, 0.049383298, 0.15750237, -0.11478398, 0.0498765, -0.01771576, 0.093446225, 0.019006833, -2.2464048E-4, -0.061441038, 0.097847626, 0.004283281, 0.053706992, 0.009988454, -0.15014431, 0.14166535, 0.09751435, 0.102532476, 0.005308849, 0.13271774, -0.059878107, 0.0221993, 0.117135644, -0.011334801, -0.024710264, -0.023027897, -0.016130574, -0.046666972, 0.014624121, 0.08421323, 0.06692702, 0.025923515, 0.0222732, 0.06477591, -0.0113320025, -0.08613905, 0.050319705, -0.004136385, -0.044563565, 0.1515195, -0.017016126, -0.0861463, -0.07683315, 0.020288685, 0.0017801934, 0.019006534, -0.011663531, 0.032768846, -0.100950435, 0.009796022, 0.124880776, 0.008370491, -0.026330657, -0.004027821, -0.021209696, 0.039961133, 0.010805228, -0.025436739, -0.009648199, -0.025574747, 0.013366432, 0.06080755, 0.051675554, -0.11963538, 0.01771051, 0.047935985, -0.04474174, 0.11451311, 0.009928426, -0.12453619, -0.12566896, 0.096510224, -0.001942677, 0.013614137, -0.025503954, -0.10239078, -0.06484347, 0.193234, 0.04834418, 0.10714206, 0.03091593, -0.041921344, 0.06626271, 0.065262206, -0.026423577, 0.058818113, -0.026069207, 0.15361166, 0.17265041, -0.19219702, -0.04913946, 0.073286936, -0.013871239, -0.11204348, 0.0029985793, -4.790734, 0.028446784, 0.015687099, -0.08928978, 0.05284358, -0.25598767, 0.017638793, 0.04708858, -8.8494027E-4, 0.032549053, -0.0013408883, -0.08099685, 0.014094401, 0.05428197, 0.008895735, -0.052127443, -0.019123822, -0.03180861, -0.05190555, 0.09669127, 0.16983789, 0.019639824, 0.01587091, 0.05906193, -0.046349242, -0.10003088, 0.10687288, 0.0067231646, -0.019107644, 0.10458537, 0.15299408, -0.006442845, -0.08347113, 0.040234894, -0.10229683, -0.029127132, 0.010930232, -0.05726208, 0.018474119, 0.019283976, 0.070765875, 0.14076112, -0.06273048, -0.108375944, -0.047028888, -0.1143751, -0.028760418, -0.21158293, 0.043967094, 0.020462122, -0.10510801, -0.16485217, 0.06556936, -0.008671817, -5.8623287E-4, 0.029037379, -0.1293192, -0.007842674, -0.16524564, -0.020396668, -0.06659272, 0.016046695, -0.032112647, -0.040887475, 0.113839366, -0.0052908086, -0.002528982, 0.044